In [1]:
import pandas as pd
import cbpro
from datetime import datetime, timedelta

public_client = cbpro.PublicClient()

In [12]:
currencies = public_client.get_currencies()
currencies[0]

currency_ids = []

for currency in currencies:
    currency_ids.append(currency['id'])

currency_ids[:10]

['ALGO', 'DASH', 'OXT', 'ATOM', 'KNC', 'MIR', 'REP', 'ICP', 'CGLD', 'COMP']

In [16]:
# Get current prices and save into a dictionary

price_data = {}

for c in currency_ids:
    price = public_client.get_product_ticker(product_id=f"LOOM-USD")

    price_data[c] = price

price_data['ETH']

{'trade_id': 146393785,
 'price': '3300.08',
 'size': '0.0020877',
 'time': '2021-08-15T22:55:04.18967Z',
 'bid': '3300.08',
 'ask': '3300.09',
 'volume': '149749.73265754'}

In [47]:
start_date = '2021-08-08'
end_date = '2021-08-15'
granularity = 86400 #Daily

public_client.get_product_historic_rates(
        "SHIB-USD", 
        start=start_date, 
        end=end_date, 
        granularity=granularity
    )

{'message': 'NotFound'}

In [15]:
price_data['ETH']

{'trade_id': 146393075,
 'price': '3302.85',
 'size': '0.01145047',
 'time': '2021-08-15T22:53:13.651285Z',
 'bid': '3302.84',
 'ask': '3302.85',
 'volume': '149446.88745776'}

In [13]:
currencies[0]


{'id': 'ALGO',
 'name': 'Algorand',
 'min_size': '1',
 'status': 'online',
 'message': '',
 'max_precision': '0.000001',
 'convertible_to': [],
 'details': {'type': 'crypto',
  'symbol': 'A',
  'network_confirmations': 1,
  'sort_order': 93,
  'crypto_address_link': 'https://algoexplorer.io/address/{{address}}',
  'crypto_transaction_link': 'https://algoexplorer.io/tx/{{txId}}',
  'push_payment_methods': ['crypto'],
  'group_types': [],
  'display_name': '',
  'processing_time_seconds': 5,
  'min_withdrawal_amount': 0.1,
  'max_withdrawal_amount': 700000}}

In [64]:
# Get historical data

# Containers
historical_data = {}
symbol = []
time = []
low = []
high = []
opn = []
close = []
volume = []

# Parameters
start_date = '2021-08-08'
end_date = '2021-08-15'
time_delta = 300
granularity = 86400 #Daily

# For first data pull want to get maximum number of days (300 per API)
start_date = datetime.strptime(end_date, '%Y-%m-%d')
start_date = start_date - timedelta(days=time_delta)
start_date = str(start_date.year)+'-'+str(start_date.month)+'-'+str(start_date.day)

for c in currency_ids:
    historic_rates = public_client.get_product_historic_rates(
        f"{c}-USD", 
        start=start_date, 
        end=end_date, 
        granularity=granularity
    )
    if type(historic_rates) is not dict: # end if no data available
        # break
    # else:
        try:
            for day in historic_rates:
                symbol.append(c)
                time.append((datetime.fromtimestamp(day[0]) + timedelta(days=1)).strftime('%Y-%m-%d')) #%H:%M:%S
                low.append(day[1])
                high.append(day[2])
                opn.append(day[3])
                close.append(day[4])
                volume.append(day[5])
        except Exception as e:
            print(c)
            print(day[0])
            print(e)


# Create DataFrame
data = {
    'Symbol':symbol,
    'Date':time,
    'High':high,
    'Low':low,
    'Open':opn,
    'Close':close,
    'Volume':volume
}
df = pd.DataFrame(data)

In [65]:
df['Symbol'].value_counts()
df[df['Symbol'] == 'ETH']

,Symbol,Date,High,Low,Open,Close,Volume
6749,ETH,2021-08-15,3323.54,3112.96,3266.52,3310.93,162311.479275
6750,ETH,2021-08-14,3333.00,3206.96,3324.51,3266.52,118892.509069
6751,ETH,2021-08-13,3330.00,3034.52,3047.21,3324.51,190905.352074
6752,ETH,2021-08-12,3239.69,2980.00,3162.16,3047.21,243214.901488
6753,ETH,2021-08-11,3275.00,3122.22,3142.18,3162.61,194586.917864
...,...,...,...,...,...,...,...
7044,ETH,2020-10-24,417.00,407.40,409.55,412.58,68349.199078
7045,ETH,2020-10-23,419.48,401.51,414.64,409.52,124844.520581
7046,ETH,2020-10-22,421.47,391.33,391.36,414.70,182333.508395
7047,ETH,2020-10-21,402.62,368.06,368.81,391.53,249849.807006
